In [24]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


#Importing libraries and data

In [38]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pylab import rcParams
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [26]:
# Load movies
movies = pd.read_csv("/content/drive/My Drive/MovieRecommender/movies.csv", header=0, parse_dates=["date"])
movies = movies.replace({np.nan: None}) # replace NaN with None


# User defined functions

In [27]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

# EDA and Data cleaning

In [28]:
movies.head()

,id,title,tagline,description,genres,keywords,date,collection,runtime,revenue,budget,director,cast,production_companies,production_countries,popularity,average_vote,num_votes,language,imdb_id,poster_url
0,862,Toy Story,None,"Led by Woody, Andy's toys live happily in his ...","animation, comedy, family","jealousy, toy, boy, friendship, friends, rival...",1995-10-30 00:00:00,Toy Story Collection,81,3.73554e+08,30000000,John Lasseter,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",Pixar Animation Studios,United States of America,21.9469,7.7,5415,en,tt0114709,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg
1,8844,Jumanji,Roll the dice and unleash the excitement!,When siblings Judy and Peter discover an encha...,"adventure, fantasy, family","board game, disappearance, based on children's...",1995-12-15 00:00:00,None,104,2.62797e+08,65000000,Joe Johnston,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...","TriStar Pictures, Teitler Film, Interscope Com...",United States of America,17.0155,6.9,2413,en,tt0113497,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg
2,15602,Grumpier Old Men,Still Yelling. Still Fighting. Still Ready for...,A family wedding reignites the ancient feud be...,"romance, comedy","fishing, best friend, duringcreditsstinger, ol...",1995-12-22 00:00:00,Grumpy Old Men Collection,101,0,0,Howard Deutch,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...","Warner Bros., Lancaster Gate",United States of America,11.7129,6.5,92,en,tt0113228,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg
3,31357,Waiting to Exhale,Friends are the people who let you be yourself...,"Cheated on, mistreated and stepped on, the wom...","comedy, drama, romance","based on novel, interracial relationship, sing...",1995-12-22 00:00:00,None,127,8.14522e+07,16000000,Forest Whitaker,"Whitney Houston, Angela Bassett, Loretta Devin...",Twentieth Century Fox Film Corporation,United States of America,3.85949,6.1,34,en,tt0114885,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg
4,11862,Father of the Bride Part II,Just When His World Is Back To Normal... He's ...,Just when George Banks has recovered from his ...,comedy,"baby, midlife crisis, confidence, aging, daugh...",1995-02-10 00:00:00,Father of the Bride Collection,106,7.65789e+07,0,Charles Shyer,"Steve Martin, Diane Keaton, Martin Short, Kimb...","Sandollar Productions, Touchstone Pictures",United States of America,8.38752,5.7,173,en,tt0113041,/e64sOI48hQXyru7naBFyssKFxVd.jpg


In [29]:
'''
# Function to display all rows and columns
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)
'''

'\n# Function to display all rows and columns\ndef display_all(df):\n    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): \n        display(df)\n'

In [30]:
display_all(movies.tail().T)

,46623,46624,46625,46626,46627
id,439050,111109,67758,227506,461257
title,Subdue,Century of Birthing,Betrayal,Satan Triumphant,Queerama
tagline,Rising and falling between a man and woman,None,A deadly game of wits.,None,None
description,Rising and falling between a man and woman.,An artist struggles to finish his work while a...,"When one of her hits goes wrong, a professiona...","In a small town live two brothers, one a minis...",50 years after decriminalisation of homosexual...
genres,"drama, family",drama,"action, drama, thriller",None,None
keywords,tragic love,"artist, play, pinoy",None,None,None
date,None,2011-11-17 00:00:00,2003-08-01 00:00:00,1917-10-21 00:00:00,2017-06-09 00:00:00
collection,None,None,None,None,None
runtime,90,360,90,87,75
revenue,0,0,0,0,0


Observations:
- Empty values in cell. Specifically,
    - Some movies do not have taglines, keywords, date collection, revenue, budget, production companies.
- Values in genre, keywords columns are stored as one string, needs to be split and stored as lists. (#Data cleaning1)

In [31]:
display_all(movies.describe(include='all').T)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,46628,NaN,NaN,NaN,109679,112908,2,26760.2,61299,159624,469172
title,46624,42276,Blackout,31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tagline,20783,20283,Which one is the first to return - memory or t...,27,NaN,NaN,NaN,NaN,NaN,NaN,NaN
description,45633,44305,No overview found.,136,NaN,NaN,NaN,NaN,NaN,NaN,NaN
genres,44104,4065,drama,5148,NaN,NaN,NaN,NaN,NaN,NaN,NaN
keywords,31739,25988,woman director,1329,NaN,NaN,NaN,NaN,NaN,NaN,NaN
date,46540,17333,2008-01-01 00:00:00,138,NaN,NaN,NaN,NaN,NaN,NaN,NaN
collection,4574,1695,Pokémon Collection,34,NaN,NaN,NaN,NaN,NaN,NaN,NaN
runtime,46360,353,90,2628,NaN,NaN,NaN,NaN,NaN,NaN,NaN
revenue,46624,6863,0,39087,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
movies.dtypes

id                       int64
title                   object
tagline                 object
description             object
genres                  object
keywords                object
date                    object
collection              object
runtime                 object
revenue                 object
budget                   int64
director                object
cast                    object
production_companies    object
production_countries    object
popularity              object
average_vote            object
num_votes               object
language                object
imdb_id                 object
poster_url              object
dtype: object

In [33]:
movies_subset = movies.dropna(subset=['keywords'])

In [34]:
print(movies.shape)
print(movies_subset.shape)

(46628, 21)
(31739, 21)


In [35]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(movies_subset.keywords)

In [36]:
tfidf_matrix.shape

(31739, 12596)

In [ ]:
tfidf_vectorizer.get_feature_names()

In [39]:
# generating the cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix,tfidf_matrix)

In [41]:
cosine_sim[0]

array([1.        , 0.02956594, 0.        , ..., 0.        , 0.        ,
       0.        ])

In [46]:
indices = pd.Series(movies_subset.title)
indices[:5]

0                      Toy Story
1                        Jumanji
2               Grumpier Old Men
3              Waiting to Exhale
4    Father of the Bride Part II
Name: title, dtype: object

In [70]:
movies_subset[movies_subset.title.str.contains('knight',case=False)==True]

,id,title,tagline,description,genres,keywords,date,collection,runtime,revenue,budget,director,cast,production_companies,production_countries,popularity,average_vote,num_votes,language,imdb_id,poster_url
165,6520,First Knight,Their greatest battle would be for her love.,The timeless tale of King Arthur and the legen...,"action, adventure, drama, romance","camelot, knight, king arthur, excalibur, knigh...",1995-07-07 00:00:00,None,134,1.276e+08,55000000,Jerry Zucker,"Sean Connery, Richard Gere, Julia Ormond, Ben ...","Columbia Pictures Corporation, First Knight Pr...",United States of America,15.1783,5.9,319,en,tt0113071,/8SRi8laZyPBdigx9qn7m3mriiRQ.jpg
323,9059,Tales from the Crypt: Demon Knight,Ready for your deadtime story?,A man on the run is hunted by a demon known as...,"horror, comedy, thriller","prostitute, key, jesus christ, chosen one, god...",1995-01-13 00:00:00,Tales from the Crypt Collection,92,0,0,Ernest R. Dickerson,"Billy Zane, William Sadler, Jada Pinkett Smith...","Universal Pictures, Tales From The Crypt Holdings",United States of America,9.45534,6.7,96,en,tt0114608,/8uD5yk0ECebJ59lhvhTBKhUojc.jpg
3524,21879,The Hollywood Knights,None,"Led by their comedic and pranking leader, Newb...",comedy,teen movie,1980-05-30 00:00:00,None,91,1e+07,2500000,Floyd Mutrux,"Robert Wuhl, Tony Danza, Fran Drescher, Michel...","PolyGram Filmed Entertainment, Casablanca Film...",United States of America,2.59922,6.4,18,en,tt0080881,/aYwg1OtY0wn6mSTHnXkFyhufQn7.jpg
3710,22167,Knightriders,The Games...The Romance...The Spirit...Camelot...,George Romero's unusual story of a modern-day ...,drama,renaissance troup,1981-04-10 00:00:00,None,145,0,3000000,George A. Romero,"Ed Harris, Gary Lahti, Tom Savini, Amy Ingerso...","Laurel Entertainment, United Film",United States of America,1.08861,6.5,17,en,tt0082622,/a0rkwzt1ARAFf8BMOHp3QHc7xyh.jpg
4205,9476,A Knight's Tale,He Will Rock You,"William Thatcher, a peasant, is sent to appren...","adventure, drama, romance, action","poetry, knight, tournament, duel, torture, wri...",2001-05-11 00:00:00,None,132,1.17487e+08,65000000,Brian Helgeland,"Heath Ledger, Rufus Sewell, Shannyn Sossamon, ...","Columbia Pictures Corporation, Escape Artists,...",United States of America,10.9006,6.6,933,en,tt0183790,/1ZUpkMivvlF0ZoyXQzHF4El3izv.jpg
4810,11469,Black Knight,It's only a flesh wound!,"Martin Lawrence plays Jamal, an employee in Me...","adventure, comedy, fantasy","england, black people, medallion, castle, time...",2001-11-21 00:00:00,None,95,0,0,Gil Junger,"Martin Lawrence, Marsha Thomason, Tom Wilkinso...","Twentieth Century Fox Film Corporation, Regenc...",United States of America,9.37044,5.1,291,en,tt0265087,/rvg5bUUQsWsB8Vbn0rFSoC6uWzH.jpg
6068,30584,Sword of the Valiant: The Legend of Sir Gawain...,None,Gawain was a squire in King Arthur's court whe...,"action, adventure, fantasy","riddle, remake, knight, king arthur",1984-08-17 00:00:00,None,102,0,0,Stephen Weeks,"Miles O'Keeffe, Leigh Lawson, Trevor Howard, S...","Golan-Globus Productions, Stephen Weeks Company",United Kingdom,1.26353,6.6,7,en,tt0084750,/5j3BUxxgVRIFlP7QfWKEI2IOL1Z.jpg
6087,6038,Shanghai Knights,A Royal Kick In The Arse.,The dynamic duo of Chon Wang and Roy O'Bannon ...,"action, adventure, comedy, western","london england, indian territory, emperor, rev...",2003-02-06 00:00:00,Shanghai Noon Collection,115,8.83235e+07,50000000,David Dobkin,"Jackie Chan, Owen Wilson, Fann Wong, Aaron Tay...","Spyglass Entertainment, Touchstone Pictures, B...","Hong Kong, United States of America",7.46978,6,707,en,tt0300471,/g5BFUJPgv9CoGDyAX4XZFcEY36a.jpg
6440,19957,Knights of the Round Table,All the glory and splendor of King Arthur's court,"In Camelot, kingdom of Arthur and Merlin, Lanc...","romance, action, adventure, fantasy, drama","holy grail, swordplay, sword, camelot, round t...",1953-12-22 00:00:00,None,115,0,0,Richard Thorpe,"Robert Taylor, Ava Gardner, Mel Ferrer, Anne C...",Metro-Goldwyn-Mayer (MGM),United States of America,4.86152,6

In [57]:
def recommendations(title, cosine_sim = cosine_sim):
    
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(movies_subset.title)[i])
        
    return recommended_movies

In [56]:
indices[indices=='Jumanji'].index[0]

1

In [71]:
recommendations('The Dark Knight')

['Toothless',
 'The Tooth Fairy',
 'Barbie Fairytopia: Magic of the Rainbow',
 'FairyTale: A True Story',
 'Rise of the Guardians',
 'The Kingdom of Fairies',
 'Tinker Bell and the Pirate Fairy',
 'Cinderella',
 'Tinker Bell and the Legend of the NeverBeast',
 'Die Salzprinzessin']